In [1]:
import numpy as np
import pandas as pd
from numpy import pi
import qiskit
from qiskit import execute, Aer
from qiskit.visualization import plot_histogram
from qiskit.tools.visualization import plot_state_city, plot_bloch_multivector
from qiskit.tools.monitor import job_monitor
from qiskit.quantum_info import state_fidelity
from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info.random.utils import random_state
from random import randint
from qiskit.providers.aer import noise
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, Aer
%config InlineBackend.figure_format = 'svg' # Makes the images look nice
from qiskit.quantum_info.operators import Operator, Pauli

# Tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
import qiskit.ignis.mitigation.measurement as mc
# imports IBMQ backends
from qiskit.providers.ibmq import IBMQ
from qiskit.providers.aqt import AQT
aqt_provider_simulators = AQT.enable_account('2633b2d14dae4d9da7c9df2503857334')
ibmq_provider = IBMQ.load_account()        # credentials stored on disk
simulator = ibmq_provider.get_backend('ibmq_qasm_simulator')
device = ibmq_provider.get_backend('ibmq_16_melbourne') #ibmq_burlington ibmq_16_melbourne
aqt_device = aqt_provider_simulators.get_backend('aqt_qasm_simulator_noise_1')
backend_IBMQ = ibmq_provider.get_backend('ibmqx2')
properties = backend_IBMQ.properties()
noise_model = noise.device.basic_device_noise_model(properties)
coupling_map = backend_IBMQ.configuration().coupling_map
basis_gates = noise_model.basis_gates

C:\Users\Niels\Envs\dev_qiskit\lib\site-packages\qiskit\providers\aer\noise\device\basic_device_model.py:112: DeprecationWarning: This function is been deprecated and moved to a method of the`NoiseModel` class. For equivalent functionality use `NoiseModel.from_backend(properties, **kwargs).
  DeprecationWarning)


In [2]:
dim = 2
z= QuantumRegister(dim)
qc_expected = QuantumCircuit(z)
qc.h(range(dim))
qreg = QuantumRegister(2)
# creating a randomized Grover operator
a = randint(0,2**dim-1)
A = np.identity(2**dim)
A[a,a] = -1
print(A)
Grov = Operator(A)
qc_expected.append(Grov,range(dim))

qc_expected.barrier()
# defining the operator to implement inversion about the mean
#MeanInv = 0.5*np.ones((4,4),float)
#np.fill_diagonal(MeanInv,-0.5)
#MeanInv = Operator(MeanInv)
#qc.append(MeanInv,[0,1])
qc_expected.h(range(dim))
qc_expected.x(range(dim))

qc_expected.cz(0,1)
#qc.h(dim-1)
#qc.cccx(0,1,2,3)
#qc.h(dim-1)
qc_expected.x(range(dim))
qc_expected.h(range(dim))
qc_expected.barrier()
#qc.measure(range(dim),range(dim))
qc_expected.draw(output='mpl')

NameError: name 'qc' is not defined

In [ ]:
state = {'XX': {'00': 0, '10': 0,'01': 0, '11': 0}, 
             'YX': {'00': 0, '10': 0,'01': 0, '11': 0}, 
             'ZX': {'00': 0, '10': 0,'01': 0, '11': 0}, 
             'XY': {'00': 0, '10': 0,'01': 0, '11': 0}, 
             'YY': {'00': 0, '10': 0,'01': 0, '11': 0}, 
             'ZY': {'00': 0, '10': 0,'01': 0, '11': 0}, 
             'XZ': {'00': 0, '10': 0,'01': 0, '11': 0}, 
             'YZ': {'00': 0, '10': 0,'01': 0, '11': 0}, 
             'ZZ': {'00': 0, '10': 0,'01': 0, '11': 0}}

In [ ]:
#simulate on qiskit aer
job = qiskit.execute(qc, Aer.get_backend('statevector_simulator'))
psi_expected = job.result().get_statevector(qc)
print('The expected state is:', psi_expected)
plot_state_city(psi_expected, title='Expected state')

In [ ]:
job = qiskit.execute(qc, device, shots=1000)
job_monitor(job)
print('Done job.')
print(job.error_message())
tom=StateTomographyFitter(job.result(), qc)

In [ ]:
state.update({'XX': {'00': state['XX']['00']+tom.data[('X','Z','Z','X')]['0000'],
                                '10': state['XX']['10']+tom.data[('X','Z','Z','X')]['1000'],
                                '01': state['XX']['01']+tom.data[('X','Z','Z','X')]['0001'],
                                '11': state['XX']['11']+tom.data[('X','Z','Z','X')]['1001']}})
state.update({'YX': {'00': state['YX']['00']+tom.data[('Y','Z','Z','X')]['0000'],
                                '10': state['YX']['10']+tom.data[('Y','Z','Z','X')]['1000'],
                                '01': state['YX']['01']+tom.data[('Y','Z','Z','X')]['0001'],
                                '11': state['YX']['11']+tom.data[('Y','Z','Z','X')]['1001']}})
state.update({'ZX': {'00': state['ZX']['00']+tom.data[('Z','Z','Z','X')]['0000'],
                                '10': state['ZX']['10']+tom.data[('Z','Z','Z','X')]['1000'],
                                '01': state['ZX']['01']+tom.data[('Z','Z','Z','X')]['0001'],
                                '11': state['ZX']['11']+tom.data[('Z','Z','Z','X')]['1001']}})
state.update({'XY': {'00': state['XY']['00']+tom.data[('X','Z','Z','Y')]['0000'],
                                '10': state['XY']['10']+tom.data[('X','Z','Z','Y')]['1000'],
                                '01': state['XY']['01']+tom.data[('X','Z','Z','Y')]['0001'],
                                '11': state['XY']['11']+tom.data[('X','Z','Z','Y')]['1001']}})
state.update({'YY': {'00': state['YY']['00']+tom.data[('Y','Z','Z','Y')]['0000'],
                                '10': state['YY']['10']+tom.data[('Y','Z','Z','Y')]['1000'],
                                '01': state['YY']['01']+tom.data[('Y','Z','Z','Y')]['0001'],
                                '11': state['YY']['11']+tom.data[('Y','Z','Z','Y')]['1001']}})
state.update({'ZY': {'00': state['ZY']['00']+tom.data[('Z','Z','Z','Y')]['0000'],
                                '10': state['ZY']['10']+tom.data[('Z','Z','Z','Y')]['1000'],
                                '01': state['ZY']['01']+tom.data[('Z','Z','Z','Y')]['0001'],
                                '11': state['ZY']['11']+tom.data[('Z','Z','Z','Y')]['1001']}})
state.update({'XZ': {'00': state['XZ']['00']+tom.data[('X','Z','Z','Z')]['0000'],
                                '10': state['XZ']['10']+tom.data[('X','Z','Z','Z')]['1000'],
                                '01': state['XZ']['01']+tom.data[('X','Z','Z','Z')]['0001'],
                                '11': state['XZ']['11']+tom.data[('X','Z','Z','Z')]['1001']}})
state.update({'YZ': {'00': state['YZ']['00']+tom.data[('Y','Z','Z','Z')]['0000'],
                                '10': state['YZ']['10']+tom.data[('Y','Z','Z','Z')]['1000'],
                                '01': state['YZ']['01']+tom.data[('Y','Z','Z','Z')]['0001'],
                                '11': state['YZ']['11']+tom.data[('Y','Z','Z','Z')]['1001']}})
state.update({'ZZ': {'00': state['ZZ']['00']+tom.data[('Z','Z','Z','Z')]['0000'],
                                '10': state['ZZ']['10']+tom.data[('Z','Z','Z','Z')]['1000'],
                                '01': state['ZZ']['01']+tom.data[('Z','Z','Z','Z')]['0001'],
                                '11': state['ZZ']['11']+tom.data[('Z','Z','Z','Z')]['1001']}})
    

In [ ]:
Iden = np.matrix('1,0,0,0;0,1,0,0;0,0,1,0;0,0,0,1')

YXPaul = np.matrix('0,0,0,0-1j;0,0,0-1j,0;0,0+1j,0,0;0+1j,0,0,0')
YYPaul = np.matrix('0,0,0,-1;0,0,1,0;0,1,0,0;-1,0,0,0')
YZPaul = np.matrix('0,0,0-1j,0;0,0,0,0+1j;0+1j,0,0,0;0,0-1j,0,0')

ZXPaul = np.matrix('0,1,0,0;1,0,0,0;0,0,0,-1;0,0,-1,0')
ZYPaul = np.matrix('0,0-1j,0,0;0+1j,0,0,0;0,0,0,0+1j;0,0,0-1j,0')
ZZPaul = np.matrix('1,0,0,0;0,-1,0,0;0,0,-1,0;0,0,0,1')
state_sim = state
# simulator Bloch vector
rxx_sim = (state_sim['XX']['00']*1 + state_sim['XX']['10']*(-1)+state_sim['XX']['01']*(-1)+state_sim['XX']['11']*1) /(state_sim['XX']['00']*1 + state_sim['XX']['10']*(1)+state_sim['XX']['01']*(1)+state_sim['XX']['11']*1)
rxy_sim = (state_sim['XY']['00']*1 + state_sim['XY']['10']*(-1)+state_sim['XY']['01']*(-1)+state_sim['XY']['11']*1) /(state_sim['XY']['00']*1 + state_sim['XY']['10']*(1)+state_sim['XY']['01']*(1)+state_sim['XY']['11']*1)
rxz_sim = (state_sim['XZ']['00']*1 + state_sim['XZ']['10']*(-1)+state_sim['XZ']['01']*(-1)+state_sim['XZ']['11']*1) /(state_sim['XZ']['00']*1 + state_sim['XZ']['10']*(1)+state_sim['XZ']['01']*(1)+state_sim['XZ']['11']*1)
ryx_sim = (state_sim['YX']['00']*1 + state_sim['YX']['10']*(-1)+state_sim['YX']['01']*(-1)+state_sim['YX']['11']*1) /(state_sim['YX']['00']*1 + state_sim['YX']['10']*(1)+state_sim['YX']['01']*(1)+state_sim['YX']['11']*1)
ryy_sim = (state_sim['YY']['00']*1 + state_sim['YY']['10']*(-1)+state_sim['YY']['01']*(-1)+state_sim['YY']['11']*1) /(state_sim['YY']['00']*1 + state_sim['YY']['10']*(1)+state_sim['YY']['01']*(1)+state_sim['YY']['11']*1)
ryz_sim = (state_sim['YZ']['00']*1 + state_sim['YZ']['10']*(-1)+state_sim['YZ']['01']*(-1)+state_sim['YZ']['11']*1) /(state_sim['YZ']['00']*1 + state_sim['YZ']['10']*(1)+state_sim['YZ']['01']*(1)+state_sim['YZ']['11']*1)
rzx_sim = (state_sim['ZX']['00']*1 + state_sim['ZX']['10']*(-1)+state_sim['ZX']['01']*(-1)+state_sim['ZX']['11']*1) /(state_sim['ZX']['00']*1 + state_sim['ZX']['10']*(1)+state_sim['ZX']['01']*(1)+state_sim['ZX']['11']*1)
rzy_sim = (state_sim['ZY']['00']*1 + state_sim['ZY']['10']*(-1)+state_sim['ZY']['01']*(-1)+state_sim['ZY']['11']*1) /(state_sim['ZY']['00']*1 + state_sim['ZY']['10']*(1)+state_sim['ZY']['01']*(1)+state_sim['ZY']['11']*1)
rzz_sim = (state_sim['ZZ']['00']*1 + state_sim['ZZ']['10']*(-1)+state_sim['ZZ']['01']*(-1)+state_sim['ZZ']['11']*1) /(state_sim['ZZ']['00']*1 + state_sim['ZZ']['10']*(1)+state_sim['ZZ']['01']*(1)+state_sim['ZZ']['11']*1)


In [ ]:
#running on a real quantum device
IBMQ.load_account()
provider = IBMQ.get_provider('ibm-q')
qcomp = provider.get_backend('ibmq_16_melbourne')
job = execute(qc, qcomp, shots=1000)
from qiskit.tools.monitor import job_monitor
job_monitor(job)
tom = StateTomographyFitter(job.result(), qc)

In [ ]:
#plotting the results of the real computation
qcounts = qres.get_counts()
print('The run counts are:', qcounts)
plot_histogram([qcounts, simcounts], legend=['Device', 'Simulator'])

In [ ]:
#noise mitigation
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, CompleteMeasFitter)

cal_circuits, state_labels = complete_meas_cal(qr = qc.qregs[0], circlabel = 'measerrormitcal')
cal_job = execute(cal_circuits,
                 backend = qcomp,
                 shots = 1024,
                 optimization_level = 0,
                 )

#this part runs the predefined calibration circuits for the circuit size and monitors the job
job_monitor(cal_job)
calres = cal_job.result()

#we use the calibration result to filter out our noisy results
meas_fitter = CompleteMeasFitter(calres, state_labels)

#filtering out noise from the quantum result
meas_fitter.plot_calibration()
meas_filter = meas_fitter.filter
mitres = meas_filter.apply(qres)

In [ ]:
#plotting the sim vs noisy results vs mitigated results
mitcounts = mitres.get_counts(qc)
plot_histogram([simcounts, qcounts, mitcounts], 
               legend = ['sim', 'noisy results', 'mitigated results'])